World Coins: A collection of coin images from 32 different currencies.

In [1]:
# Seznam knihoven
import tensorflow as tf
import matplotlib.pyplot as plt
import json
import os
import numpy as np
import cv2
import kaggle
import random
import concurrent.futures
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# Napojení dat a vytvoření cesty
data_dir_train = "data/coins/data/train/"
data_dir_val = "data/coins/data/validation/"
data_dir_test = "data/coins/data/test/"
outlier_dir = "data/coins/data/outliers"
json_path = "data/cat_to_name.json"

In [ ]:
# Načtení mapování kategorií
with open(json_path, "r") as f:
    class_mapping = json.load(f)

class_mapping

In [16]:
# Parametry pro načítání dat
img_height = 224  # Výška obrázku
img_width = 224   # Šířka obrázku
batch_size = 32   # Velikost batch

In [67]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 211
EPOCHS = 10

In [68]:
# ImageDataGenerator s odpovídajícím předzpracováním pro ResNet50
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True  # Přidáno pro lepší generalizaci rubu a líce
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [69]:
# Načtení datasetu
train_generator = train_datagen.flow_from_directory(
    data_dir_train,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    data_dir_val,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 5648 images belonging to 211 classes.
Found 844 images belonging to 211 classes.


In [ ]:
#def get_limited_generator(generator, limit):
    #for i, data in enumerate(generator):
        #if i >= limit:
            #break
        #yield data
#limited_train_generator = get_limited_generator(train_generator, limit=10)

In [ ]:
# Nastavení steps_per_epoch
steps_per_epoch = min(len(train_generator), 10)
validation_steps = len(val_generator) // BATCH_SIZE

In [80]:
# Načtení předtrénovaného modelu ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Přidání nových vrstev
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)

# Vytvoření modelu
model = Model(inputs=base_model.input, outputs=x)

# Kompilace modelu
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor='val_accuracy', mode='max')

In [ ]:
# Trénování modelu s omezeným datasetem
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint]
)

# Uložení modelu
model.save("final_resnet50_model.keras")

In [ ]:
# Postupné rozmrznutí některých vrstev pro doladění
base_model.trainable = True
for layer in base_model.layers[:100]:
    layer.trainable = False

# Rekompilace s nižší learning rate
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Další fáze trénování
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping, model_checkpoint]
)

# Uložení finálního doladěného modelu
model.save("fine_tuned_resnet50_model.keras")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Create an ImageDataGenerator instance
datagen = ImageDataGenerator(validation_split=0.2)
# Define the training data generator
train_generator = datagen.flow_from_directory(
    'path_to_directory',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Specify 'training' subset
)
# Define the validation data generator
validation_generator = datagen.flow_from_directory(
    'path_to_directory',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Specify 'validation' subset
)
# Get a smaller subset of training data
def get_limited_generator(generator, limit):
    for i, data in enumerate(generator):
        if i >= limit:
            break
        yield data
limited_train_generator = get_limited_generator(train_generator, limit=10)